# Error Per Layered Gate (EPLG)

In [ ]:
%load_ext autoreload
%autoreload 2

## Set IQM Token if using Resonance.

In [ ]:
# import os
# os.environ["IQM_TOKEN"] = "xxxxxxxxxxxxxxx"

## Choose (or define) a backend

In [ ]:
# from iqm.qiskit_iqm.iqm_provider import IQMProvider
#
# resonance_station="xxxxxxxxxxxxxxxxxxxx"
# iqm_url=f"https://cocos.resonance.meetiqm.com/{resonance_station}"
# cal_url=f"https://api.resonance.meetiqm.com/quantum-computers/v1/{resonance_station}/calibrations"
# provider = IQMProvider(iqm_url)
# backend = provider.get_backend()

In [ ]:
from iqm.benchmarks.utils import get_iqm_backend

backend = get_iqm_backend("iqmfakeapollo")

## EPLG Configuration

In [5]:
from iqm.benchmarks.randomized_benchmarking.eplg.eplg import *

In [6]:
EXAMPLE_EPLG = EPLGConfiguration(
    custom_qubits_array=[[0,1],[1,4],[4,5],[5,6],[6,11],[11,16]],
    drb_depths=sorted(list(set(np.geomspace(1, 200, num=10, endpoint=True, dtype=int).tolist())), reverse=True),
    drb_circuit_samples=25,
    shots=2**8,
    chain_length=8,
    chain_path_samples=100,
    num_disjoint_layers=2,
)

## Run the experiment

In [7]:
benchmark_eplg = EPLGBenchmark(backend, EXAMPLE_EPLG)
run_eplg = benchmark_eplg.run()

2025-03-24 14:59:26,320 - iqm.benchmarks.logging_config - INFO - WARNING: custom_qubits_array validation not yet implemented! Make sure it forms a linear chain.
2025-03-24 14:59:34,561 - iqm.benchmarks.logging_config - INFO - Clifford dictionaries imported successfully !
2025-03-24 14:59:34,563 - iqm.benchmarks.logging_config - INFO - Executing parallel Direct RB on qubits [[0, 1], [4, 5], [6, 11]] (group 1/2). Will generate and submit all 25 DRB circuits for each depth [200, 111, 61, 34, 18, 10, 5, 3, 1]
2025-03-24 14:59:34,563 - iqm.benchmarks.logging_config - INFO - Depth 200
2025-03-24 15:00:34,934 - iqm.benchmarks.logging_config - INFO - Submitting batch with 25 circuits corresponding to qubits [0, 1, 4, 5, 6, 11]
2025-03-24 15:00:34,941 - iqm.benchmarks.logging_config - INFO - Job for depth 200 submitted successfully!
2025-03-24 15:00:35,022 - iqm.benchmarks.logging_config - INFO - Depth 111
2025-03-24 15:01:12,409 - iqm.benchmarks.logging_config - INFO - Submitting batch with 25

## Perform the analysis

In [8]:
result_eplg = benchmark_eplg.analyze()

2025-03-24 15:04:58,596 - iqm.benchmarks.logging_config - INFO - Post-processing parallel Direct RB on qubits [[0, 1], [4, 5], [6, 11]].
2025-03-24 15:04:58,615 - iqm.benchmarks.logging_config - INFO - Depth 200
2025-03-24 15:04:58,658 - iqm.benchmarks.logging_config - INFO - Depth 111
2025-03-24 15:04:58,695 - iqm.benchmarks.logging_config - INFO - Depth 61
2025-03-24 15:04:58,738 - iqm.benchmarks.logging_config - INFO - Depth 34
2025-03-24 15:04:58,769 - iqm.benchmarks.logging_config - INFO - Depth 18
2025-03-24 15:04:58,797 - iqm.benchmarks.logging_config - INFO - Depth 10
2025-03-24 15:04:58,820 - iqm.benchmarks.logging_config - INFO - Depth 5
2025-03-24 15:04:58,845 - iqm.benchmarks.logging_config - INFO - Depth 3
2025-03-24 15:04:58,875 - iqm.benchmarks.logging_config - INFO - Depth 1
2025-03-24 15:04:59,026 - iqm.benchmarks.logging_config - INFO - Post-processing parallel Direct RB on qubits [[1, 4], [5, 6], [11, 16]].
2025-03-24 15:04:59,047 - iqm.benchmarks.logging_config - IN

In [9]:
run_eplg.dataset.attrs.keys()

dict_keys(['session_timestamp', 'execution_timestamp', 'backend_configuration_name', 'backend_name', 'benchmark', 'shots', 'max_gates_per_batch', 'max_circuits_per_batch', 'calset_id', 'routing_method', 'physical_layout', 'use_dd', 'dd_strategy', 'qubits_array', 'is_eplg', 'parallel_execution', 'depths', 'num_circuit_samples', 'qiskit_optim_level', 'two_qubit_gate_ensembles', 'densities_2q_gates', 'clifford_sqg_probabilities', 'sqg_gate_ensembles', 'parallel_all_0', 0, 'parallel_all_1', 1, 'backend', 'drb_depths', 'drb_circuit_samples', 'custom_qubits_array', 'chain_length', 'chain_path_samples', 'num_disjoint_layers', 'calibration_url'])

In [10]:
for o in result_eplg.observations:
    if o.name != "EPLG":
        print(f"{o.name} {o.identifier.qubit_indices}: {100*o.value:.2f} +/- {100*o.uncertainty:.2f} %")
    else:
        print(f"\n{o.name} {o.identifier.qubit_indices}: {o.value:.2e} +/- {o.uncertainty:.2e}")

average_gate_fidelity [0, 1]: 96.21 +/- 0.06 %
average_gate_fidelity [4, 5]: 96.41 +/- 0.10 %
average_gate_fidelity [6, 11]: 95.97 +/- 0.09 %
average_gate_fidelity [1, 4]: 96.42 +/- 0.10 %
average_gate_fidelity [5, 6]: 95.57 +/- 0.09 %
average_gate_fidelity [11, 16]: 95.74 +/- 0.10 %
Layer Fidelity (n_qubits=8): 78.53 +/- 0.18 %

EPLG (n_qubits=8): 3.95e-02 +/- 3.68e-04
